In [1]:
# Add user specific python libraries to path
import sys
sys.path.insert(0, "/home/smehra/local-packages")
print(sys.path)

['/home/smehra/local-packages', '', '/home/smehra/.conda/envs/smehra_py2/lib/python27.zip', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/plat-linux2', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/lib-tk', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/lib-old', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/lib-dynload', '/home/smehra/.local/lib/python2.7/site-packages', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/site-packages', '/home/smehra/.conda/envs/smehra_py2/lib/python2.7/site-packages/IPython/extensions', '/home/smehra/.ipython']


In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark.sql.functions as F


In [3]:
import os
os.environ["SPARK_CONF_DIR"] = "/data/tmp/spark/conf"

In [3]:
import pyspark
import random

from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

config = pyspark.SparkConf().setAll([('spark.ui.port', 4050), 
                                     ('spark.ui.enabled', True),
                                     
                                     # if running in local mode, driver will be only executor
                                     # hence, give driver as much memory as possible if running in local mode
                                     ('spark.driver.memory','50g'), 
                                     
                                     # set up executor config if running in cluster or client mode
                                     #('spark.executor.instances', '5'), 
                                     #('spark.executor.cores', '5'), 
                                     #('spark.executor.memory', '5g'), 
                                     #('spark.executor.memoryOverhead', '500m'),
                                     
                                     # more partitions means smaller partition size per task
                                     # hence, would reduce memory load
                                     ('spark.sql.shuffle.partitions', '1000'),
                                     
                                     # increase max result size if you are "collecting" big dataset 
                                     # driver will need more memory to collect
                                     #('spark.driver.maxResultSize', '2g'),
                                     
                                     # set location spark should use for temporary data
                                     ('spark.local.dir', '/data/tmp/smehra/tmp'),
                                     # Set location of hive database
                                     ('spark.sql.warehouse.dir', '/data/tmp/hive_warehouse'),
                                     # Add mysql connector jar to use mysql as metastore service
                                     ('spark.jars', '/data/tmp/spark/jars/mysql-connector-java-5.1.30-bin.jar'),
                                    
                                     # KryoSerializer is faster and more compact than the Java default serializer.
                                     ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
                                     
                                     # G1GC overcomes the latency and throughput limitations with the old garbage collectors.
                                     ('spark.executor.extraJavaOptions','-XX:+UseG1GC')])

spark = SparkSession.builder \
        .enableHiveSupport() \
        .config(conf=config) \
        .master("local[30]") \
        .appName("smehra_afgh_project") \
        .getOrCreate()

# Get the Hive Context
hive = HiveContext(spark.sparkContext)

spark.sparkContext._conf.getAll()


[(u'spark.driver.memory', u'50g'),
 (u'spark.sql.shuffle.partitions', u'1000'),
 (u'spark.driver.extraJavaOptions',
  u'-Dderby.system.home=/data/tmp/hive_warehouse/derby_metastore_service'),
 (u'spark.master', u'local[30]'),
 (u'spark.executor.extraJavaOptions', u'-XX:+UseG1GC'),
 (u'spark.executor.id', u'driver'),
 (u'spark.local.dir', u'/data/tmp/smehra/tmp'),
 (u'spark.serializer', u'org.apache.spark.serializer.KryoSerializer'),
 (u'spark.ui.port', u'4050'),
 (u'spark.sql.warehouse.dir', u'/data/tmp/hive_warehouse'),
 (u'spark.driver.port', u'46004'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'smehra_afgh_project'),
 (u'spark.app.id', u'local-1581446432883'),
 (u'spark.driver.host', u'umtiti.ischool.berkeley.edu'),
 (u'spark.ui.showConsoleProgress', u'true'),
 (u'spark.ui.enabled', u'True')]

In [4]:
# Read from Hive
raw_data_phone_calls = hive.sql('SELECT * FROM afghanistan.raw_data_phone_calls')
raw_data_phone_calls.show(5)

+----------------+--------+---------+----------------+---------+---------+-----------+-------------------+---------+-------------+---------+---------------+----------------+----------+------+---------+--------+---------+
|      phoneHash1|numtype1|ctrycode1|      phoneHash2| numtype2|ctrycode2|interaction|           datetime|yearmonth|call_duration|call_cost|     antenna_id|charged_duration|product_id|f_type|f_subtype|pay_type|subcos_id|
+----------------+--------+---------+----------------+---------+---------+-----------+-------------------+---------+-------------+---------+---------------+----------------+----------+------+---------+--------+---------+
|7orJ23R7GEYKqV1b|  mobile|       93|6LzVvQam1jvQMdG5|shortcode|       93|       call|2014-10-04 20:22:24|  2014-10|            9|      0.0|412204210242333|               0|        15|    SC|       SC|       0|   400003|
|9zgKqvx9LoxjQWve|  mobile|       93|LNyd29oEmzOdl8zP|   mobile|       93|       call|2014-10-04 20:22:24|  2014-10|

In [5]:
# Our methodology assumes a day starts at 5am and ends on 5am next day
# Hence, we calculate "effective" date and time respectively.

# Example:
# Actual datetime: 2013-04-21 3.40am
# Effective date: 2013-04-20
# Effective hour: 23nd hour of the day

raw_data_phone_calls_with_effective_time = raw_data_phone_calls.withColumn('effective_datetime', F.col('datetime') - F.expr("INTERVAL 5 HOURS"))


In [6]:
# keep and reformat columns needed for migration detection algorithm

raw_data_for_migration_detection = raw_data_phone_calls_with_effective_time.withColumn('date', F.date_format(F.col("effective_datetime"), "YYYYMMdd"))
                                                                                                 
raw_data_for_migration_detection = raw_data_for_migration_detection.select(F.col('phoneHash1').alias("user_id"), 
                                                                 F.col('date'), 
                                                                 F.col("antenna_id"))
raw_data_for_migration_detection.show(5)


+----------------+--------+---------------+
|         user_id|    date|     antenna_id|
+----------------+--------+---------------+
|7orJ23R7GEYKqV1b|20141004|412204210242333|
|9zgKqvx9LoxjQWve|20141004|412204110520835|
|305xqe0kZ0RVlXGg|20141004|412203810238053|
|y4rZqRp9JabjQDMK|20141004|412201010628586|
|edyL2yx10Vz0qjA8|20141004|412204110420356|
+----------------+--------+---------------+
only showing top 5 rows



In [7]:
# remove null and invalid values
raw_data_for_migration_detection_filtered = raw_data_for_migration_detection.filter(raw_data_for_migration_detection.user_id.isNotNull()
                                                                                    & (raw_data_for_migration_detection.user_id != "-99")
                                                                                    & raw_data_for_migration_detection.date.isNotNull()
                                                                                    & (raw_data_for_migration_detection.date != "-99")
                                                                                    & raw_data_for_migration_detection.antenna_id.isNotNull()
                                                                                    & (raw_data_for_migration_detection.antenna_id != -99))


In [8]:
# Load tower to antenna mapping data
tower_to_antenna_map = spark.read.csv('/data/projects/displacement_afghanistan/data/Aggregated_Groups/TowerDetails_WithGroupIDs_UTM42N.csv', header = True, inferSchema=True)
tower_to_antenna_map = tower_to_antenna_map.select(F.col('Final_Agg_GroupID').alias("tower_group_id"), F.col('callingcellid').alias("antenna_id"))
tower_to_antenna_map.show(5)


+--------------+---------------+
|tower_group_id|     antenna_id|
+--------------+---------------+
|           856|412200000000000|
|           856|412200010229701|
|           856|412200010229702|
|           856|412200010229703|
|           856|412200010229704|
+--------------+---------------+
only showing top 5 rows



In [9]:
# join daily modal location with tower to district mapping
raw_data_with_tower_groups = raw_data_for_migration_detection_filtered.join(tower_to_antenna_map, 
                                                   raw_data_for_migration_detection_filtered.antenna_id == tower_to_antenna_map.antenna_id, 
                                                   how = 'left').select(raw_data_for_migration_detection_filtered['*'], tower_to_antenna_map['tower_group_id'])
raw_data_with_tower_groups.show(5)


+----------------+--------+---------------+--------------+
|         user_id|    date|     antenna_id|tower_group_id|
+----------------+--------+---------------+--------------+
|7orJ23R7GEYKqV1b|20141004|412204210242333|           345|
|9zgKqvx9LoxjQWve|20141004|412204110520835|           123|
|305xqe0kZ0RVlXGg|20141004|412203810238053|           676|
|y4rZqRp9JabjQDMK|20141004|412201010628586|          1054|
|edyL2yx10Vz0qjA8|20141004|412204110420356|            69|
+----------------+--------+---------------+--------------+
only showing top 5 rows



## User Daily Unique Towers

In [33]:
raw_data_with_tower_locations = raw_data_with_tower_groups.select(F.col('user_id'), 
                                                       F.col("date"),
                                                       F.col("tower_group_id").alias('location'))
raw_data_with_tower_locations.show(5)


+----------------+--------+--------+
|         user_id|    date|location|
+----------------+--------+--------+
|7orJ23R7GEYKqV1b|20141004|     345|
|9zgKqvx9LoxjQWve|20141004|     123|
|305xqe0kZ0RVlXGg|20141004|     676|
|y4rZqRp9JabjQDMK|20141004|    1054|
|edyL2yx10Vz0qjA8|20141004|      69|
+----------------+--------+--------+
only showing top 5 rows



In [ ]:
## drop duplicates and sort data
raw_data_with_tower_groups_deduped = raw_data_with_tower_locations.dropDuplicates()
raw_data_with_tower_groups_deduped_sorted = raw_data_with_tower_groups_deduped.sort(["user_id", "date", "location"])

# save in hive
raw_data_with_tower_groups_deduped_sorted.write.saveAsTable('afghanistan.user_daily_unique_towers_long')

user_daily_unique_towers_long = hive.sql('SELECT * FROM afghanistan.user_daily_unique_towers_long')

In [ ]:
# convert long from to wide form dataset
# one row per user
# one column per day
# value of each cell represent all towers user used on that day

# note: collect_set dedups location values.
# use collect_list if you need *all* location values for a day for a user
user_daily_unique_towers_wide = user_daily_unique_towers_long.groupby('user_id').pivot('date').agg(F.collect_set('location'))
user_daily_unique_towers_wide.show(100)


In [ ]:
# a list of day series i.e from 20130101 to 20171231
daySeriesList = set()

for year in range(2013, 2018):
    for month in [1, 3, 5, 7, 8, 10, 12]:
        for day in range(1, 32):
            daySeriesList.add(str(year) + ("%02d"%month) + ("%02d"%day))
    
    for month in [4, 6, 9, 11]:
        for day in range(1, 31):
            daySeriesList.add(str(year) + ("%02d"%month) + ("%02d"%day))
            
    for day in range(1, 30):
        if(day == 29):
            if(year%4 == 0):
                daySeriesList.add(str(year) + "02" + ("%02d"%day))
        else:
            daySeriesList.add(str(year) + "02" + ("%02d"%day))


In [ ]:
# add empty columns for days for which we did not have any users making any calls

# existing list of columns in user_daily_unique_towers_wide table
existingDaySeriesColumns = user_daily_unique_towers_wide.columns
existingDaySeriesColumns.remove('user_id')
existingDaySeriesColumns = set(existingDaySeriesColumns)

missingColumns = daySeriesList.difference(existingDaySeriesColumns)
print('missing columns: ', missingColumns)

for newColumn in missingColumns:
    user_daily_unique_towers_wide = user_daily_unique_towers_wide.withColumn(str(newColumn), F.array())
    print('added column: ', newColumn)

In [ ]:
# save in hive
user_daily_unique_towers_wide.write.saveAsTable('afghanistan.user_daily_unique_towers_wide')

## User Daily Unique Districts

In [10]:
# Load tower to district mapping data
tower_to_district_map = spark.read.csv('/data/projects/displacement_afghanistan/data/Aggregated_Groups/Final_Aggregated_GroupIDs_UTM42N.csv', header = True, inferSchema=True)
tower_to_district_map = tower_to_district_map.select(F.col('Final_Agg_GroupID').alias("tower_group_id"), F.col('distid').alias("district_id"))
tower_to_district_map.show(5)


+--------------+-----------+
|tower_group_id|district_id|
+--------------+-----------+
|             0|       2008|
|             1|       2008|
|             2|       2007|
|             3|       2008|
|             4|       2008|
+--------------+-----------+
only showing top 5 rows



In [11]:
raw_data_with_districts = raw_data_with_tower_groups.join(tower_to_district_map, 
                                                   raw_data_with_tower_groups.tower_group_id == tower_to_district_map.tower_group_id, 
                                                   how = 'left').select(raw_data_with_tower_groups['*'], tower_to_district_map['district_id'])

raw_data_with_districts = raw_data_with_districts.select(F.col('user_id'), 
                                                       F.col("date"),
                                                       F.col("district_id").alias('location'))
raw_data_with_districts.show(5)



+----------------+--------+--------+
|         user_id|    date|location|
+----------------+--------+--------+
|7orJ23R7GEYKqV1b|20141004|    1701|
|9zgKqvx9LoxjQWve|20141004|    2001|
|305xqe0kZ0RVlXGg|20141004|    2918|
|y4rZqRp9JabjQDMK|20141004|     101|
|edyL2yx10Vz0qjA8|20141004|    2001|
+----------------+--------+--------+
only showing top 5 rows



In [ ]:
## drop duplicates and sort data
raw_data_with_districts_deduped = raw_data_with_districts.dropDuplicates()
raw_data_with_districts_deduped_sorted = raw_data_with_districts_deduped.sort(["user_id", "date", "location"])

# save in hive
raw_data_with_districts_deduped_sorted.write.saveAsTable('afghanistan.user_daily_unique_districts_long')

user_daily_unique_districts_long = hive.sql('SELECT * FROM afghanistan.user_daily_unique_districts_long')

In [ ]:
# convert long from to wide form dataset
# one row per user
# one column per day
# value of each cell represent all towers user used on that day

# note: collect_set dedups location values.
# use collect_list if you need *all* location values for a day for a user

user_daily_unique_districts_wide = user_daily_unique_districts_long.groupby('user_id').pivot('date').agg(F.collect_set('location'))
user_daily_unique_districts_wide.show(100)


In [ ]:
# a list of day series i.e from 20130101 to 20171231
daySeriesList = set()

for year in range(2013, 2018):
    for month in [1, 3, 5, 7, 8, 10, 12]:
        for day in range(1, 32):
            daySeriesList.add(str(year) + ("%02d"%month) + ("%02d"%day))
    
    for month in [4, 6, 9, 11]:
        for day in range(1, 31):
            daySeriesList.add(str(year) + ("%02d"%month) + ("%02d"%day))
            
    for day in range(1, 30):
        if(day == 29):
            if(year%4 == 0):
                daySeriesList.add(str(year) + "02" + ("%02d"%day))
        else:
            daySeriesList.add(str(year) + "02" + ("%02d"%day))


In [ ]:
# add empty columns for days for which we did not have any users making any calls

# existing list of columns in user_daily_unique_towers_wide table
existingDaySeriesColumns = user_daily_unique_districts_wide.columns
existingDaySeriesColumns.remove('user_id')
existingDaySeriesColumns = set(existingDaySeriesColumns)

missingColumns = daySeriesList.difference(existingDaySeriesColumns)
print('missing columns: ', missingColumns)

for newColumn in missingColumns:
    user_daily_unique_districts_wide = user_daily_unique_districts_wide.withColumn(str(newColumn), F.array())
    print('added column: ', newColumn)

In [ ]:
# save in hive
user_daily_unique_districts_wide.write.saveAsTable('afghanistan.user_daily_unique_districts_wide')

In [ ]:
spark.stop()